# 简介实现线性回归-实操笔记

这个其实相比于之前那个从零实现其实区别就在于这个是采用了torch的库的，而之前那个是没有使用太多的torch自己的求导和模型使用函数的

## 导入库--自动生成人造数据集

In [1]:
import numpy as np
import torch as tor
from torch.utils import data as tor_u_data
from d2l import torch as tor_d2l

true_w = tor.tensor([2, -3.4])
true_b = 4.2
featrues, labels = tor_d2l.synthetic_data(true_w, true_b, 1000)

这一步就是我们采用方法生成人造数据集的方法

## 调用现有API框架读取数据集

In [2]:
def load_array(data_arrays, batch_size, is_train = True):
    """构造一个PyTorch数据迭代器"""
    dataset = tor_u_data.TensorDataset(*data_arrays)                    # 将数据据转化为适合torch.utils.data进行操作的类型
    return tor_u_data.DataLoader(dataset, batch_size, shuffle=is_train) # 是一个装载器，其实就是进行之前那个洗牌操作，搞池化的操作

batch_size = 10
data_iter = load_array((featrues, labels), batch_size)

next(iter(data_iter))                                                   # next是返回下一个迭代对象，而iter是生成迭代对象

[tensor([[-0.5885,  0.5512],
         [-0.0603,  0.1598],
         [ 0.4254, -1.0167],
         [ 0.5744,  0.5323],
         [-0.8593,  0.5564],
         [-1.2283, -1.1211],
         [-0.2989,  0.5893],
         [-1.0152,  0.8133],
         [ 0.9207, -1.2001],
         [-1.8483,  1.3435]]),
 tensor([[ 1.1303],
         [ 3.5304],
         [ 8.5162],
         [ 3.5365],
         [ 0.5911],
         [ 5.5554],
         [ 1.5925],
         [-0.5994],
         [10.1119],
         [-4.0785]])]

## 使用框架预定义好的层

In [3]:
# 'nn' 是一个神经网络的缩写neural network
from torch import nn as tor_nn

net = tor_nn.Sequential(tor_nn.Linear(2,1)) # Sequential是作为一个容器，就是说取装不同结构的圣经网络所进行的一个容器，里面可以一层层装不同的神经网络。Linear就是线性回归神经网络2是输入的单个数据的维度，1是输出维度

### 初始化模型参数

In [4]:
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

tensor([0.])

### 损失函数计算

#### 均方误差使用MSELoss 【平方 $L_2$ 范数】

In [5]:
loss = tor_nn.MSELoss()

#### 实例化SGD(随机下降找最好)

In [6]:
trainer = tor.optim.SGD(net.parameters(), lr=0.03) # optim 就是optimizer 优化器

## 开始训练啦

In [10]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X), y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(featrues), labels)
    print(f'epoch {epoch + 1}, loss{l:f}') # l 是L的小写

epoch 1, loss0.000102
epoch 2, loss0.000103
epoch 3, loss0.000102


In [9]:
w = net[0].weight.data
print('w的估计误差：', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('b的估计误差：', true_b - b)
w, b

w的估计误差： tensor([-0.0007, -0.0003])
b的估计误差： tensor([-7.6771e-05])


(tensor([[ 2.0007, -3.3997]]), tensor([4.2001]))